In [1]:
%pip install ultralytics==8.2.103 -q
%pip install supervision==0.24.0 -q
%pip install face_recognition opencv-python numpy
%pip install face_recognition --no-deps

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ultralytics import YOLO
from IPython.display import display, Image
from IPython.display import clear_output
from datetime import datetime
from flask import Flask, render_template, Response
import yaml
import supervision as sv
import ultralytics
import cv2
import sqlite3
import os
import atexit
import numpy as np
import face_recognition
import time

# Path model dan database
MODEL_PATH = "model/best.pt"
DB_PATH = "presensi.db"
FACES_DIR = "faces" 

In [3]:
import cv2 
camera = cv2.VideoCapture()
camera.release()

In [3]:
# Load wajah-wajah yang dikenali dari folder 'faces'
known_face_encodings = []
known_face_names = []

for filename in os.listdir(FACES_DIR):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        path = os.path.join(FACES_DIR, filename)
        image = face_recognition.load_image_file(path)
        encodings = face_recognition.face_encodings(image)
        if encodings:
            known_face_encodings.append(encodings[0])
            known_face_names.append(os.path.splitext(filename)[0])
print(f"[INFO] Loaded {len(known_face_encodings)} known faces.")


[INFO] Loaded 4 known faces.


In [4]:
import cv2
import face_recognition
import sqlite3
from flask import Flask, Response, jsonify
from datetime import datetime

# ==============================
# Konfigurasi Database
# ==============================
conn = sqlite3.connect("presensi.db", check_same_thread=False)
c = conn.cursor()
c.execute("""
CREATE TABLE IF NOT EXISTS presensi (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nama TEXT,
    status TEXT,
    waktu TEXT
)
""")
conn.commit()

def reset_presensi():
    c.execute("DELETE FROM presensi")
    conn.commit()

def sudah_terdeteksi(nama):
    row = c.execute("SELECT 1 FROM presensi WHERE nama=? AND status='Hadir'", (nama,)).fetchone()
    return row is not None

def tandai_hadir(nama):
    waktu = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ada = c.execute("SELECT 1 FROM presensi WHERE nama=? AND status='Hadir'", (nama,)).fetchone()
    if not ada:
        c.execute("INSERT INTO presensi (nama, status, waktu) VALUES (?, 'Hadir', ?)", (nama, waktu))
        conn.commit()

def get_status_presensi():
    daftar_anak = [
        "Jonea Kristiawan",
        "Kevin Jovani Kuslin",
        "Kevin Tanwiputra",
        "Adriel Bernhard Tanuhariono"
    ]
    rows = c.execute("SELECT nama FROM presensi WHERE status='Hadir'").fetchall()
    hadir_set = {row[0] for row in rows}
    hasil = []
    for nama in daftar_anak:
        status = "Hadir" if nama in hadir_set else "Belum Hadir"
        hasil.append({"nama": nama, "status": status})
    return hasil


In [6]:
# Daftar anak-anak
daftar_anak = ["Adriel Bernhard T", "Jonea Kristiawan", "Kevin Tanwiputra", "Kevin Jiovanni Kuslin"]

# Load YOLOv8-OBB model
MODEL_PATH = "model/best.pt"
model = YOLO(MODEL_PATH)

def get_status_presensi(daftar_anak):
    c.execute("SELECT nama, waktu FROM presensi WHERE status = 'Hadir'")
    hasil = c.fetchall()
    nama_hadir = set(nama for nama, _ in hasil)
    tidak_hadir = [nama for nama in daftar_anak if nama not in nama_hadir]
    return hasil, tidak_hadir

In [7]:
import cv2
from flask import Flask, Response, render_template, jsonify, request 

app = Flask(__name__)

# Buka kamera sekali (global)
camera = cv2.VideoCapture(0) 

def gen():
    while True:
        success, frame = camera.read()
        if not success:
            break

        results = model.predict(source=frame, conf=0.75, stream=True)

        for r in results:
            if r.obb is not None and hasattr(r.obb, "xywhr"):
                cls_list = r.obb.cls.cpu().numpy().astype(int)
                names = model.names

                for cls in cls_list:
                    nama_terdeteksi = names[cls] if cls in names else f"Unknown-{cls}"
                    if nama_terdeteksi in daftar_anak:
                        if not sudah_terdeteksi(nama_terdeteksi):
                            tandai_hadir(nama_terdeteksi)
                            print(f"[INFO] {nama_terdeteksi} TERDETEKSI - disimpan ke database")

            # Tambah overlay
            annotated_frame = r.plot()

            _, buffer = cv2.imencode('.jpg', annotated_frame) 
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


@app.route('/')
def index():
    # Ambil nama yang sudah hadir dari database
    rows = c.execute("SELECT nama FROM presensi WHERE status='Hadir' ORDER BY waktu DESC").fetchall()
    nama_list = [row[0] for row in rows]
    
    # Render template dan kirim nama_list
    return render_template('html/index.html', nama_list=nama_list)


@app.route('/video_feed')
def video_feed():
    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route("/api/status_presensi")
def status_presensi():
    rows = c.execute("SELECT nama, status FROM presensi ORDER BY waktu DESC").fetchall()
    return jsonify([{"nama": r[0], "status": r[1]} for r in rows])


@app.route("/reset_presensi", methods=["POST"])
def reset_presensi_route():
    c.execute("DELETE FROM presensi")  # hapus semua data presensi
    conn.commit()
    return "OK", 200

@app.route('/student')
def student():
    return render_template("html/studentlog.html") 

@app.route("/api/update_status", methods=["POST"])
def update_status():
    data = request.get_json()
    nama = data.get("nama")
    status = data.get("status")
    waktu = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Cek apakah sudah ada record untuk nama ini
    row = c.execute("SELECT id FROM presensi WHERE nama=?", (nama,)).fetchone()
    if row:
        # Update status
        c.execute("UPDATE presensi SET status=?, waktu=? WHERE id=?", (status, waktu, row[0]))
    else:
        # Insert baru
        c.execute("INSERT INTO presensi (nama, status, waktu) VALUES (?, ?, ?)", (nama, status, waktu))
    conn.commit()
    return jsonify({"success": True})


if __name__ == "__main__":
    try:
        # Jalankan Flask
        app.run(host="0.0.0.0", port=5000, debug=True, use_reloader=False)
    finally:
        # Kamera dilepas hanya saat server dihentikan (Ctrl+C)
        if camera.isOpened():
            camera.release()
            print("✅ Kamera dilepas dengan aman")


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://174.139.114.179:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Sep/2025 19:35:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 19:35:53] "GET /static/css/global.css HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 19:35:53] "GET /static/assets/ataim2.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 19:35:53] "GET /static/js/index.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 19:35:53] "GET /static/assets/font/Jura-Medium.ttf HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 19:35:53] "GET /api/presensi HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2025 19:35:53] "GET /api/status_presensi HTTP/1.1" 200 -


127.0.0.1 - - [08/Sep/2025 19:35:55] "GET /api/recent_attendees HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2025 19:35:56] "GET /api/recent_attendees HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2025 19:35:56] "GET /api/status_presensi HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2025 19:35:58] "GET /api/recent_attendees HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2025 19:35:59] "GET /api/recent_attendees HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2025 19:35:59] "GET /api/status_presensi HTTP/1.1" 200 -


0: 480x640 (no detections), 552.1ms


127.0.0.1 - - [08/Sep/2025 19:36:00] "GET /video_feed HTTP/1.1" 200 -


...

0: 480x640 341.0ms
Speed: 7.3ms preprocess, 341.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 292.5ms
Speed: 7.6ms preprocess, 292.5ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 301.3ms
Speed: 8.5ms preprocess, 301.3ms inference, 11.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 330.8ms
Speed: 7.1ms preprocess, 330.8ms inference, 7.9ms postprocess per image at shape (1, 3, 480, 640)



127.0.0.1 - - [08/Sep/2025 19:36:07] "GET /api/recent_attendees HTTP/1.1" 404 -


0: 480x640 379.1ms
Speed: 5.2ms preprocess, 379.1ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 374.8ms
Speed: 8.3ms preprocess, 374.8ms inference, 16.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 313.9ms
Speed: 12.6ms preprocess, 313.9ms inference, 9.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 310.3ms
Speed: 8.1ms preprocess, 310.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)



127.0.0.1 - - [08/Sep/2025 19:36:08] "GET /api/recent_attendees HTTP/1.1" 404 -
127.0.0.1 - - [08/Sep/2025 19:36:08] "GET /api/status_presensi HTTP/1.1" 200 -


0: 480x640 324.5ms
Speed: 10.4ms preprocess, 324.5ms inference, 13.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 284.5ms
Speed: 2.9ms preprocess, 284.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 289.0ms
Speed: 9.5ms preprocess, 289.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 276.5ms
Speed: 4.9ms preprocess, 276.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 366.5ms
✅ Kamera dilepas dengan aman


Full Length Code untuk Face Recogniton Dlib OBB

In [ ]:
from ultralytics import YOLO
from datetime import datetime
from flask import Flask, render_template, Response
import cv2
import sqlite3
import os
import numpy as np
import face_recognition

# Path model dan database
MODEL_PATH = "model/best.pt" 
FACES_DIR = "faces" 
DB_PATH = "presensi.db"

# Load wajah-wajah yang dikenali dari folder 'faces'
known_face_encodings = []
known_face_names = []

for filename in os.listdir(FACES_DIR):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        path = os.path.join(FACES_DIR, filename)
        image = face_recognition.load_image_file(path)
        encodings = face_recognition.face_encodings(image)
        if encodings:
            known_face_encodings.append(encodings[0])
            known_face_names.append(os.path.splitext(filename)[0])
print(f"[INFO] Loaded {len(known_face_encodings)} known faces.")

import cv2
import face_recognition
import sqlite3
from flask import Flask, Response, jsonify
from datetime import datetime

# ==============================
# Konfigurasi Database
# ==============================
conn = sqlite3.connect("presensi.db", check_same_thread=False)
c = conn.cursor()
c.execute("""
CREATE TABLE IF NOT EXISTS presensi (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nama TEXT,
    status TEXT,
    waktu TEXT
)
""")
conn.commit()

def reset_presensi():
    c.execute("DELETE FROM presensi")
    conn.commit()

def sudah_terdeteksi(nama):
    row = c.execute("SELECT 1 FROM presensi WHERE nama=? AND status='Hadir'", (nama,)).fetchone()
    return row is not None

def tandai_hadir(nama):
    waktu = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ada = c.execute("SELECT 1 FROM presensi WHERE nama=? AND status='Hadir'", (nama,)).fetchone()
    if not ada:
        c.execute("INSERT INTO presensi (nama, status, waktu) VALUES (?, 'Hadir', ?)", (nama, waktu))
        conn.commit()

def get_status_presensi():
    daftar_anak = [
        "Jonea Kristiawan",
        "Kevin Jovani Kuslin",
        "Kevin Tanwiputra",
        "Adriel Bernhard Tanuhariono"
    ]
    rows = c.execute("SELECT nama FROM presensi WHERE status='Hadir'").fetchall()
    hadir_set = {row[0] for row in rows}
    hasil = []
    for nama in daftar_anak:
        status = "Hadir" if nama in hadir_set else "Belum Hadir"
        hasil.append({"nama": nama, "status": status})
    return hasil

# Daftar anak-anak
daftar_anak = ["Adriel Bernhard T", "Jonea Kristiawan", "Kevin Tanwiputra", "Kevin Jiovanni Kuslin"]

# Load YOLOv8-OBB model
model = YOLO(MODEL_PATH)

def get_status_presensi(daftar_anak):
    c.execute("SELECT nama, waktu FROM presensi WHERE status = 'Hadir'")
    hasil = c.fetchall()
    nama_hadir = set(nama for nama, _ in hasil)
    tidak_hadir = [nama for nama in daftar_anak if nama not in nama_hadir]
    return hasil, tidak_hadir 

import cv2
from flask import Flask, Response, render_template, jsonify, request 

app = Flask(__name__)

# Buka kamera sekali (global)
camera = cv2.VideoCapture(0) 

def gen():
    while True:
        success, frame = camera.read()
        if not success:
            break

        results = model.predict(source=frame, conf=0.75, stream=True)

        for r in results:
            if r.obb is not None and hasattr(r.obb, "xywhr"):
                cls_list = r.obb.cls.cpu().numpy().astype(int)
                names = model.names

                for cls in cls_list:
                    nama_terdeteksi = names[cls] if cls in names else f"Unknown-{cls}"
                    if nama_terdeteksi in daftar_anak:
                        if not sudah_terdeteksi(nama_terdeteksi):
                            tandai_hadir(nama_terdeteksi)
                            print(f"[INFO] {nama_terdeteksi} TERDETEKSI - disimpan ke database")

            # Tambah overlay
            annotated_frame = r.plot()

            _, buffer = cv2.imencode('.jpg', annotated_frame) 
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    # Ambil nama yang sudah hadir dari database
    rows = c.execute("SELECT nama FROM presensi WHERE status='Hadir' ORDER BY waktu DESC").fetchall()
    nama_list = [row[0] for row in rows]
    
    # Render template dan kirim nama_list
    return render_template('html/index.html', nama_list=nama_list)

@app.route('/video_feed')
def video_feed():
    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route("/api/status_presensi")
def status_presensi():
    rows = c.execute("SELECT nama, status FROM presensi ORDER BY waktu DESC").fetchall()
    return jsonify([{"nama": r[0], "status": r[1]} for r in rows])


@app.route("/reset_presensi", methods=["POST"])
def reset_presensi_route():
    c.execute("DELETE FROM presensi")  # hapus semua data presensi
    conn.commit()
    return "OK", 200

@app.route('/student')
def student():
    return render_template("html/studentlog.html") 

@app.route("/api/update_status", methods=["POST"])
def update_status():
    data = request.get_json()
    nama = data.get("nama")
    status = data.get("status")
    waktu = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Cek apakah sudah ada record untuk nama ini
    row = c.execute("SELECT id FROM presensi WHERE nama=?", (nama,)).fetchone()
    if row:
        # Update status
        c.execute("UPDATE presensi SET status=?, waktu=? WHERE id=?", (status, waktu, row[0]))
    else:
        # Insert baru
        c.execute("INSERT INTO presensi (nama, status, waktu) VALUES (?, ?, ?)", (nama, status, waktu))
    conn.commit()
    return jsonify({"success": True})


if __name__ == "__main__":
    try:
        # Jalankan Flask
        app.run(host="0.0.0.0", port=7860, debug=True, use_reloader=False)
    finally:
        # Kamera dilepas hanya saat server dihentikan (Ctrl+C)
        if camera.isOpened():
            camera.release()
            print("✅ Kamera dilepas dengan aman")

In [ ]:
def gen():
    global frame_counter
    while True:
        success, frame = camera.read()
        if not success:
            break
        frame_counter += 1

        # lakukan prediksi YOLO (fast)
        try:
            results = model(frame, conf=DETECT_CONF)  # non-stream mode
        except Exception:
            # jika error pada model, debug dan continue
            print("[ERROR] YOLO predict error:")
            traceback.print_exc()
            results = []

        # untuk setiap result, ambil boxes (mendukung r.boxes ataupun r.obb)
        for r in results:
            # Handle axis-aligned boxes (r.boxes) jika ada
            if hasattr(r, "boxes") and r.boxes is not None and len(r.boxes) > 0:
                for box in r.boxes:
                    try:
                        xyxy = box.xyxy[0].cpu().numpy().astype(int)  # [x1,y1,x2,y2]
                        x1, y1, x2, y2 = int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])
                    except Exception:
                        continue

                    roi = safe_crop(frame, x1, y1, x2, y2)
                    if roi is None or roi.size == 0:
                        continue

                    # hanya lakukan embedding setiap beberapa frame agar tidak berat
                    do_identify = (frame_counter % DEEPFACE_FRAME_SKIP == 0)

                    detected_name = None
                    if do_identify and len(known_face_data) > 0:
                        try:
                            # convert BGR->RGB, ensure uint8
                            roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB).astype(np.uint8)
                            emb = DeepFace.represent(img_path=roi_rgb, model_name="Facenet", enforce_detection=False)
                            if emb:
                                embedding_frame = np.array(emb[0]["embedding"], dtype=np.float32)
                                detected_name = match_face_embedding(embedding_frame)
                        except Exception:
                            # jangan crash, log dan lanjut
                            print("[WARN] DeepFace error on ROI:")
                            traceback.print_exc()
                            detected_name = None

                    # If throttling: jika sudah terdeteksi belakangan, gunakan last_seen
                    display_name = "Unknown"
                    if detected_name:
                        now = datetime.now()
                        last = last_seen.get(detected_name)
                        if last is None or (now - last).total_seconds() > DETECTION_COOLDOWN:
                            # update DB only if belum terdeteksi
                            if not sudah_terdeteksi(detected_name):
                                tandai_hadir(detected_name)
                                print(f"[INFO] {detected_name} TERDETEKSI - disimpan ke database")
                            last_seen[detected_name] = now
                        display_name = detected_name

                    # gambar bounding box dan label
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, display_name, (x1, max(20, y1-10)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

            # Handle OBB jika model OBB (opsional)
            if hasattr(r, "obb") and r.obb is not None and hasattr(r.obb, "xywhr"):
                try:
                    # ambil first obb xywhr -> xywhr (x_center, y_center, w, h, angle)
                    xywhr = r.obb.xywhr[0].cpu().numpy().astype(float)
                    xc, yc, w_box, h_box, ang = xywhr
                    x1 = int(xc - w_box/2); y1 = int(yc - h_box/2)
                    x2 = int(xc + w_box/2); y2 = int(yc + h_box/2)
                except Exception:
                    continue

                roi = safe_crop(frame, x1, y1, x2, y2)
                if roi is None or roi.size == 0:
                    continue

                # sama proses identifikasi seperti di atas (skip untuk ringkas)
                display_name = "Unknown"
                if (frame_counter % DEEPFACE_FRAME_SKIP == 0) and len(known_face_data) > 0:
                    try:
                        roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB).astype(np.uint8)
                        emb = DeepFace.represent(img_path=roi_rgb, model_name="Facenet", enforce_detection=False)
                        if emb:
                            embedding_frame = np.array(emb[0]["embedding"], dtype=np.float32)
                            detected_name = match_face_embedding(embedding_frame)
                            if detected_name:
                                now = datetime.now()
                                last = last_seen.get(detected_name)
                                if last is None or (now - last).total_seconds() > DETECTION_COOLDOWN:
                                    if not sudah_terdeteksi(detected_name):
                                        tandai_hadir(detected_name)
                                        print(f"[INFO] {detected_name} TERDETEKSI - disimpan ke database")
                                    last_seen[detected_name] = now
                                display_name = detected_name
                    except Exception:
                        pass

                # gambar kotak sederhana
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, display_name, (x1, max(20, y1-10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

        # overlay presensi (kanan/atas)
        daftar_anak = sorted(list(known_face_data.keys()))
        cv2.rectangle(frame, (10, 10), (520, 10 + 30 + 25 * len(daftar_anak)), (0, 0, 0), -1)
        cv2.putText(frame, "Presensi Siswa", (20, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        y_offset = 60
        for nama in daftar_anak:
            status = "Hadir" if sudah_terdeteksi(nama) else "Belum Hadir"
            warna = (0,255,0) if status == "Hadir" else (0,0,255)
            cv2.putText(frame, f"{nama} - {status}", (20, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.55, warna, 2)
            y_offset += 25

        # encode & yield
        _, buffer = cv2.imencode('.jpg', frame)
        frame_bytes = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')